In [7]:
import cognify

In [8]:
#================================================================
# Evaluator
#================================================================

# metric = cognify.metric.F1Str()
lm_config = cognify.llm.LMConfig(
    model='gpt-4o-mini',
)
metric = cognify.metric.AgentJudge(
    criterion="The given output should have the same statement as the ground truth.",
    model_config=lm_config,
    need_ground_truth=True,
)

In [3]:
#================================================================
# Data Loader
#================================================================

import json

def load_data_minor():
    with open("data._json", "r") as f:
        data = json.load(f)
          
    # format to (input, output) pairs
    new_data = []
    for d in data:
        input = (d["question"], d["docs"])
        output = d["answer"]
        new_data.append((input, output))
    return new_data[:5], None, new_data[5:]

In [4]:
#================================================================
# Optimizer Set Up
#================================================================
from cognify.hub.search import default

search_settings = default.create_search(n_trials=5)

In [5]:
train, val, test = load_data_minor()

In [ ]:
opt_cost, pareto_frontier, opt_logs = cognify.optimize(
    script_path="cognify_workflow.py",
    control_param=search_settings,
    train_set=train,
    val_set=val,
    eval_fn=metric,
)

In [ ]:
new_workflow = cognify.load_workflow(config_id='Pareto_1', opt_result_path='opt_results')

In [ ]:
input = (
    "What was the 2010 population of the birthplace of Gerard Piel?", 
    [
        'Gerard Piel | Gerard Piel (1 March 1915 in Woodmere, N.Y. – 5 September 2004) was the publisher of the new Scientific American magazine starting in 1948. He wrote for magazines, including "The Nation", and published books on science for the general public. In 1990, Piel was presented with the "In Praise of Reason" award by the Committee for Skeptical Inquiry (CSICOP).',
        'Woodmere, New York | Woodmere is a hamlet and census-designated place (CDP) in Nassau County, New York, United States. The population was 17,121 at the 2010 census.',
    ],
)

new_workflow(input)

In [ ]:
eval_result = cognify.evaluate(
    config_id='Pareto_1',
    test_set=test,
    opt_result_path='opt_results',
    n_parallel=10,
    eval_fn=metric,
)

In [ ]:
cognify.inspect(
    'opt_results'
)

In [6]:
eval_result = cognify.evaluate(
    config_id='NoChange',
    test_set=test,
    workflow='cognify_workflow.py',
    n_parallel=10,
    eval_fn=metric,
)

> Evaluation in evaluate_raw | (avg score: 0.00, avg cost@1000: 0.00 $):   0%|          | 0/10 [00:00<?, ?it/s…

----- Testing Raw Program -----
=========== Evaluation Results ===========
  Quality: 0.100, Cost per 1K invocation ($): 0.16 $
